In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense  #全连接层
from keras.optimizers import RMSprop
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import boston_housing

In [3]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()

57344/57026 [==============================] - 0s 7us/step


In [6]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(404, 13) (404,)
(102, 13) (102,)


标准化处理
$X_i = \frac{X_i-mean}{std}$

In [8]:
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train-mean)/std
x_test = (x_test-mean)/std

In [13]:
def build_model():
    model = Sequential()
    model.add(Dense(64,input_shape=(13,),activation="relu"))
    model.add(Dense(64,activation="relu"))
    model.add(Dense(1))
    model.compile(optimizer = RMSprop(),loss = "mse",metrics=["mae"])
    return model

In [16]:
k =4
num_val_samples = len(x_train)//k
num_iter =10
num_scores = []

### 交叉验证训练模型

<img src = "img/4.png">

In [17]:
for i in range(k):
    val_data = x_train[i*num_val_samples :(i+1)*num_val_samples]
    val_targes = y_train[i*num_val_samples :(i+1)*num_val_samples]
    
    part_train_data = np.concatenate(
    [ x_train[:i *num_val_samples],
      x_train[(i+1)*num_val_samples:]], 
      axis=0)
    
    part_train_targes =np.concatenate(
    [ y_train[:i *num_val_samples],
      y_train[(i+1)*num_val_samples:]], 
      axis=0)
    
    model = build_model()
    model.fit(part_train_data,part_train_targes,epochs=num_iter,batch_size=1,verbose=0)
    
    val_mse,val_mae = model.evaluate(val_data,val_targes,verbose=0)
    num_scores.append(val_mae)
    print("第",i+1,"MSE:",val_mse,"MAE",val_mae)

第 1 MSE: 12.197622790194973 MAE 2.1889097690582275
第 2 MSE: 12.618929324763837 MAE 2.697058916091919
第 3 MSE: 16.27091397861443 MAE 2.5228919982910156
第 4 MSE: 26.65988747436221 MAE 3.4304490089416504


In [24]:
test_model = build_model()
test_model.fit(x_train,y_train,epochs=80,batch_size=16,verbose = 0)

In [25]:
 val_mse,val_mae = test_model.evaluate(x_test,y_test,verbose=0)

In [26]:
val_mse

16.977018767712163